In [127]:
import pandas as pd
import glob
import tqdm as tqdm

In [129]:
def calc_props(df, label):
    #index with largest area will always be the spheroid
    #indices with area = 0 are protrusions
    #others are disconnected regions

    core = df[df.Area == df.Area.max()]
    core_centroid = (float(core.X),float(core.Y))
    core_area = float(core.Area)

    discr = df.loc[df['Area'] < core_area]
    discr = discr.loc[discr['Area'] > 0]
    protrusions = df.loc[df['Area'] == 0]

    def dist_calc(x0,y0,x1,y1):
        return pow(((x0-x1)*(x0-x1)+(y0-y1)*(y0-y1)),.5)

    protrusion_dists = [dist_calc(centroid[0],centroid[1],i[0],i[1]) for i in protrusions.loc[:, ['X', 'Y']].to_numpy()]
    discr_dists = [dist_calc(centroid[0],centroid[1],i[0],i[1]) for i in discr.loc[:, ['X', 'Y']].to_numpy()]

    protrusion_num = len(protrusions)
    protrusion_max = max(protrusion_dists)
    protrusion_mean = sum(protrusion_dists)/len(protrusion_dists)
    discr_num = len(discr)
    discr_max = max(discr_dists)
    discr_mean = sum(discr_dists)/len(discr_dists)

    discr_mean_area = sum(discr.loc[:,['Area']].to_numpy())/len(discr)

    return dict(label = label,
                area = core_area, 
                centroid = core_centroid, 
                perimeter = float(core['Perim.']),
                circularity = float(core['Circ.']),
                aspect_ratio = float(core['AR']),
                roundness = float(core['Round']),
                solidity = float(core['Solidity']),
                protrusion_num = protrusion_num,
                protrusion_mean_len = protrusion_mean,
                protrusion_max_len = protrusion_max,
                discr_num = discr_num,
                discr_mean_dist = discr_mean,
                discr_max_dist = discr_max,
                discr_mean_area = discr_mean_area,
                )


In [138]:
globalpath = "content/sample_data" #folder path

results = []
for path in tqdm.tqdm(sorted(glob.glob(globalpath+'/*.csv')),position = 0, leave = True, desc = globalpath):
    results.append(calc_props(pd.read_csv(path),path))

pd.DataFrame.from_dict(results).to_csv('out.csv', index=False)

content/sample_data: 100%|██████████| 6/6 [00:00<00:00, 125.17it/s]
